In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0004}
band = [5]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.058118                    1 -0.040577
0.000750        2  0.078796                    2 -0.021103
0.001052        3  0.141390                    3  0.025463
0.001476        4  0.211531                    4  0.079027
0.002070        5  0.287482                    5  0.139411
0.002904        6  0.368151                    6  0.206424
0.004074        7  0.451561                    7  0.281510
0.005714        8  0.534892                    8  0.364552
0.008015        9  0.614633                    9  0.453043
0.011243       10  0.686432                   10  0.544629
0.015771       11  0.745496                   11  0.635981
0.022122       12  0.787649                   12  0.722252
0.031031       13  0.810169                   13  0.795941
0.043528       14  0.792132                   14  0.829705
0.061057       15  0.720423                   15  0.798329
0.085645       16  0.627359                   16  0.716295
0.120136       17  0.541204                   17  0.606247
0.168516       18  0.469413                   18  0.487370
0.236378       19  0.417222                   19  0.387723
0.331549       20  0.391370                   20  0.336837
0.465100       21  0.407800                   21  0.358370
0.652400       22  0.364736                   22  0.340022
0.915100       23  0.256676                   23  0.254699
1.283650       24  0.184515                   24  0.193400
1.800600       25  0.143089                   25  0.152439
2.525700       26  0.112057                   26  0.118703
3.542800       27  0.084788                   27  0.091584
4.969550       28  0.065987                   28  0.074726
6.970850       29  0.057454                   29  0.068784
9.778100       30  0.050825                   30  0.062078
13.715850      31  0.042472                   31  0.050221
19.239350      32  0.032976                   32  0.034927
26.987250      33  0.028533                   33  0.025380
37.855300      34  0.029459                   34  0.023414
53.100050      35  0.029553                   35  0.024608
73.887500      36  0.029254                   36  0.028517
97.662500      37  0.028859                   37  0.031216
121.437500     38  0.026439                   38  0.029824
145.212500     39  0.022978                   39  0.026083
168.987500     40  0.020016                   40  0.022253
192.762500     41  0.017013                   41  0.018179
216.537500     42  0.013773                   42  0.013846
240.312500     43  0.010077                   43  0.008979
264.087500     44  0.005370                   44  0.003105
287.862500     45 -0.001114                   45 -0.004295
311.637500     46 -0.003347                   46 -0.007104
335.412500     47  0.002515                   47 -0.001511
359.187500     48  0.010272                   48  0.006377
382.962500     49  0.016711                   49  0.013112
406.737500     50  0.022659                   50  0.019282
430.512500     51  0.028386                   51  0.025138
454.287500     52  0.033977                   52  0.030800
478.062500     53  0.039465                   53  0.036495
501.837500     54  0.044845                   54  0.042102
525.612500     55  0.050089                   55  0.047584
549.387500     56  0.055132                   56  0.053053
573.162500     57  0.059885                   57  0.058400
596.937500     58  0.064242                   58  0.063484
620.712500     59  0.068093                   59  0.068083
644.487500     60  0.071361                   60  0.072204
668.262500     61  0.074022                   61  0.075650
692.037500     62  0.076144                   62  0.078579
715.812500     63  0.077848                   63  0.081078
739.587500     64  0.079315                   64  0.083301
763.362500     65  0.080759                   65  0.085445
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -16.429054  0.000000 -16.429054 -16.326106  1.898246e-09   
0.000624    2     -16.429086  0.000036 -16.429050 -16.326135  2.521061e-05   
0.000876    3     -16.429098  0.000051 -16.429047 -16.326146  3.591602e-05   
0.001229    4     -16.429116  0.000074 -16.429042 -16.326162  5.264493e-05   
0.001723    5     -16.429139  0.000110 -16.429029 -16.326183  7.874805e-05   
0.002417    6     -16.429171  0.000166 -16.429006 -16.326212  1.193417e-04   
0.003391    7     -16.429214  0.000251 -16.428963 -16.326251  1.822458e-04   
0.004757    8     -16.429271  0.000381 -16.428890 -16.326303  2.798613e-04   
0.006672    9     -16.429345  0.000576 -16.428769 -16.326372  4.310331e-04   
0.009359    10    -16.429439  0.000866 -16.428573 -16.326460  6.635030e-04   
0.013128    11    -16.429557  0.001291 -16.428267 -16.326571  1.017767e-03   
0.018415    12    -16.429701  0.001901 -16.427800 -16.326706  1.551509e-03   
0.025830    13    -16.429868  0.002760 -16.427108 -16.326864  2.343709e-03   
0.036232    14    -16.430047  0.003937 -16.426110 -16.327035  3.495064e-03   
0.050823    15    -16.430225  0.005485 -16.424740 -16.327207  5.101260e-03   
0.071291    16    -16.430403  0.007409 -16.422994 -16.327379  7.209199e-03   
0.100000    17    -16.430577  0.009716 -16.420860 -16.327544  9.810947e-03   
0.140271    18    -16.430715  0.012437 -16.418278 -16.327673  1.283168e-02   
0.196760    19    -16.430761  0.015624 -16.415137 -16.327707  1.612785e-02   
0.275997    20    -16.430599  0.019377 -16.411221 -16.327539  1.959913e-02   
0.387100    21    -16.429959  0.023888 -16.406070 -16.326903  2.339636e-02   
0.543100    22    -16.427987  0.029453 -16.398535 -16.324915  2.803143e-02   
0.761700    23    -16.424388  0.035298 -16.389090 -16.321194  3.311602e-02   
1.068500    24    -16.420204  0.040442 -16.379762 -16.316783  3.796323e-02   
1.498800    25    -16.415927  0.045570 -16.370356 -16.312193  4.323181e-02   
2.102400    26    -16.411575  0.051449 -16.360125 -16.307436  4.937597e-02   
2.949000    27    -16.407252  0.058364 -16.348888 -16.302622  5.646652e-02   
4.136600    28    -16.403412  0.066452 -16.336960 -16.298155  6.488505e-02   
5.802500    29    -16.400425  0.076487 -16.323938 -16.294344  7.582198e-02   
8.139200    30    -16.398018  0.089983 -16.308035 -16.290834  9.135349e-02   
11.417000   31    -16.396184  0.107883 -16.288301 -16.287614  1.122401e-01   
16.014700   32    -16.395861  0.130692 -16.265169 -16.285754  1.377339e-01   
22.464000   33    -16.399567  0.159591 -16.239976 -16.288117  1.667827e-01   
31.510500   34    -16.409110  0.199710 -16.209400 -16.296677  2.025435e-01   
44.200100   35    -16.424041  0.258923 -16.165118 -16.310444  2.515089e-01   
62.000000   36    -16.445711  0.342906 -16.102804 -16.330844  3.238007e-01   
85.775000   37    -16.475025  0.454611 -16.020414 -16.359223  4.325002e-01   
109.550000  38    -16.504632  0.565492 -15.939139 -16.388679  5.498786e-01   
133.325000  39    -16.536534  0.671856 -15.864678 -16.421160  6.663629e-01   
157.100000  40    -16.573214  0.773248 -15.799966 -16.459040  7.777092e-01   
180.875000  41    -16.615181  0.871587 -15.743594 -16.502869  8.842162e-01   
204.650000  42    -16.663035  0.967354 -15.695680 -16.553452  9.860020e-01   
228.425000  43    -16.717653  1.060761 -15.656892 -16.612385  1.083933e+00   
252.200000  44    -16.780443  1.151933 -15.628511 -16.681409  1.178248e+00   
275.975000  45    -16.854411  1.241023 -15.613388 -16.763632  1.269217e+00   
299.750000  46    -16.946469  1.329946 -15.616524 -16.865382  1.358868e+00   
323.525000  47    -17.057272  1.431323 -15.625950 -16.986144  1.459621e+00   
347.300000  48    -17.177751  1.558884 -15.618867 -17.116120  1.585343e+00   
371.075000  49    -17.301950  1.712011 -15.5899

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')